# Installs

In [ ]:
# Folders you will need
# mkdir Processing
# TRAIN_FOLDERPATH = './Processing/crowd_processed/'
# mkdir crowd_processed
# TEST_FOLDERPATH = './Processing/crowd_processed_expert/'
# mkdir crowd_processed_expert

In [ ]:
# Uncomment to install libraries you might not have
# !python3 -m pip install --upgrade nbconvert 
# !python3 -m pip install --upgrade nbstripout 
# !python3 -m pip install tomotopy
# !python3 -m pip install sklearn
# !python3 -m pip install imblearn

# Imports

In [ ]:
import nltk
# Things to install from nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
%load_ext autoreload
%autoreload 2

import tomotopy as tp
from itertools import chain
import tqdm
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle
import os.path

import sklearn.preprocessing
import imblearn.over_sampling

import dataloader
import bow
import slda
import post_classifier
import aggregate
import user_classifier

# Get Data

In [ ]:
def process_data(POSTPATH, LABELPATH, USERPATH, FOLDERPATH, subset = 1000, append_title = False, 
                 filter_images=True, filter_direction=False, filter_control=False):
    print('START: Processing data')
    users = dataloader.load_user_subset_from_train(USERPATH, subset = subset)
    
    user_to_post, post_to_words, post_to_metadata = dataloader.load_posts(POSTPATH, user_subset = users, append_title = append_title)
    post_to_label = dataloader.load_classification(LABELPATH, user_to_post, post_to_words, post_to_metadata, user_subset = users)
    filtered_data, sw_posts, sw_timestamps = dataloader.filter_posts(post_to_label, post_to_metadata, filter_images=filter_images)
    print(len(filtered_data))
    filtered_data = dataloader.filter_near_SW(filtered_data,post_to_metadata, sw_timestamps,
                                             filter_direction=filter_direction, filter_control=filter_direction)
    print(len(filtered_data))

    filtered_data = dataloader.filter_stopwords(filtered_data)
    sw_posts = dataloader.filter_stopwords(sw_posts)
    
    dataloader.save_to_folder(FOLDERPATH, user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps)
    print('DONE: Processing data')
    return user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps

def load_processed_data(FOLDERPATH):
    print('IN PROGRESS: Loading data')
    return dataloader.load_from_folder(FOLDERPATH)

In [ ]:
def print_label_size(data):
    a_len = len([data[key] for key in data.keys() if data[key][2] == 'a'])
    b_len = len([data[key] for key in data.keys() if data[key][2] == 'b'])
    c_len = len([data[key] for key in data.keys() if data[key][2] == 'c'])
    d_len = len([data[key] for key in data.keys() if data[key][2] == 'd'])
    total_len = len(data)
    control_len = total_len - a_len - b_len - c_len - d_len
    print('a: ', a_len)
    print('b: ', b_len)
    print('c: ', c_len)
    print('d: ', d_len)
    print('controls: ', control_len)
    print('total: ', total_len)

# Feature Extraction

In [ ]:
def get_slda(data, data_type='filter_normal', filter_direction=False, filter_control=False, num_topics=40):
    slda_model_filename = 'slda_'+data_type+'_'+str(num_topics)+'_filter_direction='+str(filter_direction)+'_filter_control='+str(filter_control)+'_model.bin'
    slda_vectors_filename = 'slda_'+data_type+'_'+str(num_topics)+'_filter_direction='+str(filter_direction)+    '_filter_control='+str(filter_control)+'_vectors.pickle'

    if os.path.exists(slda_model_filename):
        print('IN PROGRESS: Loading sLDA model')
        slda_model = tp.SLDAModel.load(slda_model_filename)
    else:
        print('START: Training sLDA model')
        slda_model = slda.train_slda_model_from_data(data, topics=num_topics)
        slda_model.save(slda_model_filename)
        print('IN PROGRESS: Saving sLDA model')
    # print out topics and top n words
#     get_topics(slda_model)

    slda_vectors = extract_slda_features(data, slda_model, slda_vectors_filename, d_type='train')

    return slda_model, slda_vectors

In [ ]:
def get_lda(data, data_type='filter_normal', filter_direction=False, filter_control=False, num_topics=40):
    lda_model_filename = 'lda_'+data_type+'_'+str(num_topics)+'_filter_direction='+str(filter_direction)+'_filter_control='+str(filter_control)+'_model.bin'
    lda_vectors_filename = 'lda_'+data_type+'_'+str(num_topics)+'_filter_direction='+str(filter_direction)+    '_filter_control='+str(filter_control)+'_vectors.pickle'

    if os.path.exists(lda_model_filename):
        print('IN PROGRESS: Loading LDA model')
        lda_model = tp.LDAModel.load(lda_model_filename)
    else:
        print('START: Training LDA model')
        lda_model = slda.train_lda_model_from_data(data, topics=num_topics)
        lda_model.save(lda_model_filename)
        print('IN PROGRESS: Saving LDA model')
    # print out topics and top n words
    get_topics(lda_model, sLDA=False)

    lda_vectors = extract_slda_features(data, lda_model, lda_vectors_filename, d_type='train')

    return lda_model, lda_vectors

In [ ]:
def get_bow(data, filter_direction=False, filter_control=False, data_type='filter_normal'):
    pca_model_filename = 'train_pca_'+data_type+'_filter_direction='+str(filter_direction)+'_filter_control='+str(filter_control)+'_model.pickle'
    bow_vectors_filename = 'train_bow_'+data_type+'_filter_direction='+str(filter_direction)+'_filter_control='+str(filter_control)+'_vectors.pickle'
    word2index_filename = 'train_bow_word'+'_filter_direction='+str(filter_direction)+'_filter_control='+str(filter_control)+'_word2index.pickle'
    index2word_filename = 'train_bow_word'+'_filter_direction='+str(filter_direction)+'_filter_control='+str(filter_control)+'_index2word.pickle'

    if os.path.exists(pca_model_filename) and os.path.exists(bow_vectors_filename):
        print('IN PROGRESS: Loading PCA model')
        with open(pca_model_filename, 'rb') as f:
            pca_model = pickle.load(f)
        with open(bow_vectors_filename, 'rb') as f:
            bow_vectors = pickle.load(f)
        with open(word2index_filename, 'rb') as f:
            word2index = pickle.load(f)
        with open(index2word_filename, 'rb') as f:
            index2word = pickle.load(f)
    else:
        print('START: Training BOW model')
        word2index, index2word = bow.generate_vocabulary(data)
        pca_model, bow_vectors = bow.get_PCA_vectors_from_post_set(data, word2index)
        print('IN PROGRESS: Saving BOW model')
        with open(pca_model_filename, 'wb') as f:
            pickle.dump(pca_model, f)
        with open(bow_vectors_filename, 'wb') as f:
            pickle.dump(bow_vectors, f)
        with open(word2index_filename, 'wb') as f:
            pickle.dump(word2index, f)
        with open(index2word_filename, 'wb') as f:
            pickle.dump(index2word, f)
    return pca_model, bow_vectors, word2index, index2word 

In [ ]:
def train_feature_model(data, data_type='filter_normal', sLDA=False, LDA=False, BOW=False, 
                        empath=False, filter_direction=False, filter_control=False, num_topics=40):
    slda_model = ''
    vectors = ''
    pca_model = ''
    bow_vectors = ''
    word2index = ''
    index2word = ''
    if sLDA and BOW:
        model, vectors = get_slda(data, data_type=data_type, filter_direction=filter_direction,
                                              num_topics=num_topics)
        pca_model, bow_vectors, word2index, index2word = get_bow(data, data_type=data_type, 
                                                                   filter_direction=filter_direction,
                                                                   filter_control=filter_control)
    elif sLDA:
        model, vectors = get_slda(data, data_type=data_type, num_topics=num_topics, 
                                              filter_direction=filter_direction,
                                              filter_control=filter_control)
    elif LDA:
        model, vectors = get_slda(data, data_type=data_type, num_topics=num_topics, 
                                           filter_direction=filter_direction, 
                                           filter_control=filter_control)
    elif BOW:
        pca_model, bow_vectors, word2index, index2word = get_bow(data, data_type=data_type,
                                                                  filter_direction=filter_direction,
                                                                  filter_control=filter_control)
    elif empath:
        vectors = extract_empath_features(data)
    else:
        print('Please choose an option')
    return model, vectors, pca_model, bow_vectors, word2index, index2word

In [ ]:
def extract_slda_features(data, slda_model, slda_vectors_filename, d_type='train'):
    slda_vectors_filename = d_type+'_'+slda_vectors_filename
    
    if os.path.exists(slda_vectors_filename):
        with open(slda_vectors_filename, 'rb') as f:
            print('IN PROGRESS: Loading sLDA vectors')
            slda_vectors=pickle.load(f)
    else:
        print('START: Getting topic sLDA vecs')
        slda_vectors = slda.get_topic_vecs(slda_model, data)
        print('IN PROGRESS: Saving sLDA vectors')
        with open(slda_vectors_filename, 'wb') as f:
            pickle.dump(slda_vectors, f)
    return slda_vectors

def extract_lda_features(data, lda_model, lda_vectors_filename, d_type='train'):

    lda_vectors_filename = d_type+'_'+lda_vectors_filename
    
    if os.path.exists(lda_vectors_filename):
        with open(lda_vectors_filename, 'rb') as f:
            print('IN PROGRESS: Loading LDA vectors')
            lda_vectors=pickle.load(f)
    else:
        print('START: Getting topic LDA vecs')
        lda_vectors = slda.get_topic_vecs(lda_model, data)
        print('IN PROGRESS: Saving sLDA vectors')
        with open(lda_vectors_filename, 'wb') as f:
            pickle.dump(lda_vectors, f)
    return lda_vectors

def extract_bow_features(data, word2index, pca_model, data_type='filter_normal', 
                         filter_direction=False, filter_control=False):
    bow_vectors_filename = 'test_bow_'+data_type+'_filter_direction='+str(filter_direction)+'_filter_control='+str(filter_control)+'_vectors.pickle'
    
    if os.path.exists(bow_vectors_filename):
        with open(bow_vectors_filename, 'rb') as f:
            print('IN PROGRESS: Loading BOW vectors')
            bow_vectors=pickle.load(f)
    else:
        print('START: Getting BOW dev vecs')
        _, bow_vectors = bow.get_PCA_vectors_from_post_set(data, word2index, pca_model=pca_model)
        print('IN PROGRESS: Saving BOW dev vectors')
        with open(bow_vectors_filename, 'wb') as f:
            pickle.dump(bow_vectors, f)
    
    return bow_vectors

def extract_empath_features(data, data_type='filter_normal', filter_direction=False, 
                            filter_control=False, d_type='train'):

    empath_vectors_filename = d_type+'_empath_'+data_type+'_filter_direction='+str(filter_direction)+'_filter_control='+str(filter_control)+'_vectors.pickle'
    
    if os.path.exists(empath_vectors_filename):
        with open(empath_vectors_filename, 'rb') as f:
            print('IN PROGRESS: Loading empath vectors')
            empath_vectors=pickle.load(f)
    else:
        print('START: Getting empath vecs')
        empath_vectors = empath_extractor.get_empath_vectors_from_post_set(data)
        print('IN PROGRESS: Saving empath vectors')
        with open(empath_vectors_filename, 'wb') as f:
            pickle.dump(empath_vectors, f)
    
    return empath_vectors

In [ ]:
def get_topics(model, sLDA=True):
    if sLDA:
        slda_coefficients = model.get_regression_coef(0)
        data = []
        for k in range(model.k):
            top_words = model.get_topic_words(k, top_n=40)
            words = [word for (word, float) in top_words]
            words = ", ".join(words)
            data.append([words, slda_coefficients[k]])

        indices = np.array(slda_coefficients).argsort()
        data = np.array(data)
        data = data[indices]

        topics = pd.DataFrame(data, columns=["Topic", "Suicidality Coefficient"])
    else:
        for k in range(model.k):
            top_words = model.get_topic_words(k, top_n=40)
            words = [word for (word, float) in top_words]
            words = ", ".join(words)
            data.append([words])
        
        data = np.array(data)
        topics = pd.DataFrame(data, columns=["Topic"])
            
    print(topics)

In [ ]:
def slda_vectorize_data(model, FOLDERPATH):
    X, y, post_vectors = slda.vectorize_data_set(model, FOLDERPATH)
    return  X, y, post_vectors
    
def minmax_norm(arr):    
    return (arr - np.min(arr))/(np.max(arr) -np.min(arr))

def format_features(FOLDERPATH, model='', vectors='', pca_model='', bow_vectors='', num_topics=0,
                    data_type='filter_normal', filter_direction=False, filter_control=False,
                    sLDA=False, LDA=False, BOW=False, empath=False, d_type='train'):
    post_vectors = ''
    fil = '_filter_direction='+str(filter_direction)+'_filter_control='+str(filter_control)
    if sLDA and BOW:
        X_file = '_X_'+data_type+'_sLDA='+str(sLDA)+'_'+str(num_topics)+'_BOW='+str(BOW)+fil+'.pickle'
        y_file = '_y_'+data_type+'_sLDA='+str(sLDA)+'_'+str(num_topics)+'_BOW='+str(BOW)+fil+'.pickle'
        post_vectors_file = 'post_vectors_'+data_type+'_sLDA='+str(sLDA)+'_'+str(num_topics)+'_BOW='+str(BOW)+fil+'.pickle'
    elif sLDA:
        X_file = '_X_'+data_type+'_sLDA='+str(sLDA)+'_'+str(num_topics)+fil+'.pickle'
        y_file = '_y_'+data_type+'_sLDA='+str(sLDA)+'_'+str(num_topics)+fil+'.pickle'
        post_vectors_file = 'post_vectors_'+data_type+'_sLDA='+str(sLDA)+'_'+str(num_topics)+fil+'.pickle'
    elif BOW:
        X_file = '_X_'+data_type+'_BOW='+str(BOW)+fil+'.pickle'
        y_file = '_y_'+data_type++'_BOW='+str(BOW)+fil+'.pickle'
        post_vectors_file = 'post_vectors_'+data_type+'_BOW='+str(BOW)+fil+'.pickle'
    elif empath:
        X_file = '_X_'+data_type+'_empath='+str(empath)+fil+'.pickle'
        y_file = '_y_'+data_type++'_empath='+str(empath)+fil+'.pickle'
        post_vectors_file = 'post_vectors_'+data_type+'_empath='+str(empath)+fil+'.pickle'
        
    X_file = d_type+X_file
    y_file = d_type+y_file
    post_vectors_file = d_type+post_vectors_file
        
    if os.path.exists(X_file) and os.path.exists(y_file) and os.path.exists(post_vectors_file):
        with open(X_file,'rb') as f:
            X = pickle.load(f)
        with open(y_file,'rb') as f:
            y = pickle.load(f)
        with open(post_vectors_file,'rb') as f:
            post_vectors = pickle.load(f)
    else:
        if sLDA and BOW:
            print('START: Vectorize sLDA and BOW')
            X = np.array([np.concatenate([minmax_norm(vectors[key][0]),minmax_norm(bow_vectors[key][0])]) for key in vectors.keys()])
            y = np.array([vectors[key][1] for key in vectors.keys()])
            y = y.reshape(np.shape(y)[0])
        elif BOW:
            print('START: Vectorize BOW')
            X = np.array([bow_vectors[key][0] for key in bow_vectors.keys()])
            y = np.array([bow_vectors[key][1] for key in bow_vectors.keys()])
            y = y.reshape(np.shape(y)[0])
        elif sLDA:
            print('START: Vectorize sLDA')
            X, y, post_vectors = slda_vectorize_data(model, FOLDERPATH)
            y = y.reshape(np.shape(y)[0])
        elif LDA:
            print('START: Vectorize LDA')
            X, y, post_vectors = lda_vectorize_data(model, FOLDERPATH)
            y = y.reshape(np.shape(y)[0])
        elif empath:
            print('START: Vectorize empath')
            X = np.array([vectors[key][0] for key in vectors.keys()])
            y = np.array([vectors[key][1] for key in vectors.keys()])
            y = y.reshape(np.shape(y)[0])
        else:
            print('Oop! please pick a feature extractor, i.e. sLDA=True')
        if d_type=='train':
            ros = imblearn.over_sampling.RandomOverSampler(random_state=0)
            X, y = ros.fit_resample(X, y)
            
        print('DONE: Vectorizing')
        print('IN PROGRESS: Saving vectors')
        with open(X_file,'wb') as f:
            pickle.dump(X, f)
        with open(y_file,'wb') as f:
            pickle.dump(y, f)
        with open(post_vectors_file,'wb') as f:
            pickle.dump(post_vectors, f)
        
    return X, y, post_vectors

# Post-level Classification Train

In [ ]:
def train_post_classifier(X_train, y_train, post_clf_type='RbfSVM', num_topics=40, 
                          data_type='filter_normal', filter_direction=False, filter_control=False,
                          sLDA=False, LDA=False, BOW=False, empath=False):
    fil = '_filter_direction='+str(filter_direction)+'_filter_control='+str(filter_control)
    if sLDA and BOW:
        file = 'p_clf_'+data_type+'_sLDA='+str(sLDA)+'_'+str(num_topics)+'_BOW='+str(BOW)+fil+'_'+post_clf_type+'.pickle'
    elif sLDA:
        file = 'p_clf_'+data_type+'_sLDA='+str(sLDA)+'_'+str(num_topics)+fil+'_'+post_clf_type+'.pickle'
    elif BOW:
        file = 'p_clf_'+data_type+'_BOW='+str(BOW)+fil+'_'+post_clf_type+'.pickle'
    elif empath:
        file = 'p_clf_'+data_type+'_empath='+str(empath)+fil+'_'+post_clf_type+'.pickle'
        
    if os.path.exists(file):
        print('IN PROGRESS: Loading post classifier')
        with open(file, 'rb') as f:
            p_clf = pickle.load(f)
    else:
        print('START: Training post classifier')
#         post_clf_types = ['LogReg', 'LinearSVM', 'RbfSVM', 'AdaBoost', 'RandomForest', 'MLP']
#         if post_clf_types[0] == post_clf_type:
            #param_dict = {'C':[0.2,0.5,0.7,1,1.5,2,5]}
#         elif post_clf_types[1] == post_clf_type:
            #param_dict = {'C':[0.2,0.5,1,2]}
        #p_clf = post_classifier.PostClassification(post_clf_type)
#         elif post_clf_types[2] == post_clf_type:
            #param_dict = {'C':[0.5,1,2,5]}           
#         elif post_clf_types[5] == post_clf_type:
            #param_dict = {'hidden_layer_sizes':[(64,64),(64,64,64),(32,32), (32,32,32)], 'learning_rat
        #p_clf.train_grid_search_CV(X_train, y_train, param_dict, groups=5)

        
        p_clf = post_classifier.PostClassification(post_clf_type)
        p_clf.train(X_train, y_train)
        print('IN PROGRESS: Saving post classifier')
        with open(file, 'wb') as f:
            pickle.dump(p_clf, f)
    return p_clf

# Post-level Classification Test

In [ ]:
def predict_posts(p_clf, X_test, y_test, print_metrics=True):
    y_pred = p_clf.test(X_test)
    if print_metrics:
        p_clf.get_metrics(y_test, y_pred)
    return y_pred

# User-level Classifcation Test

In [ ]:
def format_post_labels(data, post_vectors, y_pred, FOLDERPATH, sLDA=False, BOW=False):
    # relabel the user labels from 1 for d and 0 for all else
    user_to_y = defaultdict(int)
    for post_id in tqdm.tqdm(data.keys()):
        user_to_y[data[post_id][0]] = (1 if data[post_id][2] == 'd' else 0)
    
    # format the data to get user for each post
    post2user_ypred = defaultdict(list)
    for i, post_id in enumerate(post_vectors.keys()):
        user_id = data[post_id][0]
        post2user_ypred[post_id] = [user_id, y_pred[i]]
        
    user_to_post_label = aggregate.aggregate_posts(FOLDERPATH, post2user_ypred)
    
    return user_to_post_label, user_to_y

In [ ]:
def format_user_pred(user_to_y, user_to_ypred):
    user_y = []
    user_y_pred = []
    for user_id in user_to_ypred:
        user_y.append(user_to_y[user_id])
        user_y_pred.append(user_to_ypred[user_id])
    return user_y, user_y_pred

def predict_users(user_to_post_label, user_to_y, user_clf_type='Max'):
    u_clf = user_classifier.UserClassification(user_to_post_label)
    if user_clf_type=='Max':
        user_to_ypred = u_clf.argmax()
    elif user_clf_type=='Threshold':
        user_to_ypred = u_clf.find_threshold(user_to_y)
    elif user_clf_type=='Minimum':
        user_to_ypred = u_clf.minimum(1)
    
    user_y, user_ypred = format_user_pred(user_to_y, user_to_ypred)
    print(u_clf.get_metrics(user_y, user_ypred))

# Wrapped

In [ ]:
def wrapped(train_data_processed=True, dev_data_processed=True, sLDA=False, 
            data_type='filter_normal', num_topics=40, BOW=False, LDA=False, empath=False, 
            filter_direction=False, filter_control=False,
            post_clf_type='RbfSVM', user_clf_types=['Max']):
    # Make sure you have created TRAIN_FOLDERPATH and DEV_FOLDERPATH directories on local
    TRAIN_POSTPATH = './Data/crowd/train/shared_task_posts.csv'
    TRAIN_LABELPATH = './Data/crowd/train/crowd_train.csv'
    TRAIN_USERPATH = './Data/crowd/train/task_C_train.posts.csv'
    
    DEV_POSTPATH = './Data/crowd/test/shared_task_posts_test.csv'
    DEV_LABELPATH = './Data/crowd/test/crowd_test_C.csv'
    DEV_USERPATH = './Data/crowd/test/task_C_test.posts.csv'
    
    TRAIN_FOLDERPATH = './Processing/crowd_processed/'
    DEV_FOLDERPATH = './Processing/crowd_processed_test/'
    
    # Proccess or load processed training data
    if not train_data_processed:
        user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps = process_data(TRAIN_POSTPATH, TRAIN_LABELPATH, 
                                                                                              TRAIN_USERPATH, TRAIN_FOLDERPATH)
    else:
        user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps = load_processed_data(TRAIN_FOLDERPATH)
    
    # print class sizes
    print_label_size(filtered_data)
    
    model, vectors, pca_model, bow_vectors, word2index, index2word  = train_feature_model(filtered_data, BOW=BOW,
                                                                                                    sLDA=sLDA, LDA=LDA,
                                                                                                    empath=empath, 
                                                                                                    data_type=data_type, 
                                                                                                    num_topics=num_topics,
                                                                                                    filter_direction=filter_direction, 
                                                                                                    filter_control=filter_direction)
    
    X_train, y_train, post_vectors = format_features(TRAIN_FOLDERPATH, model=model, vectors=vectors, 
                                                          pca_model=pca_model, bow_vectors=bow_vectors,
                                                         data_type=data_type, num_topics=num_topics, d_type='train', 
                                                         BOW=BOW, sLDA=sLDA, LDA=LDA, empath=empath,
                                                        filter_direction=filter_direction, filter_control=filter_direction)

#     y_train = y_train.reshape(np.shape(y_train)[0])
    
    p_clf = train_post_classifier(X_train, y_train, post_clf_type=post_clf_type, num_topics=num_topics, 
                                  data_type=data_type, BOW=BOW, sLDA=sLDA, LDA=LDA, empath=empath,
                                  filter_direction=filter_direction, filter_control=filter_direction)
    
    print('DONE: Training complete')
    
    
#     Proccess or load processed development data
    if not dev_data_processed:
        user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps = process_data(DEV_POSTPATH, 
                                                                                              DEV_LABELPATH, DEV_USERPATH, DEV_FOLDERPATH)
    else:
        user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps = load_processed_data(DEV_FOLDERPATH)
    
    print_label_size(filtered_data)
    
    
    if sLDA:
        slda_vectors_filename = 'slda_'+data_type+'_'+str(num_topics)+'_filter_direction='+str(filter_direction)+'_filter_control='+str(filter_control)+'_vectors.pickle'
        vectors = extract_slda_features(filtered_data, model, slda_vectors_filename, d_type='dev')
    if LDA:
        lda_vectors_filename = 'lda_'+data_type+'_'+str(num_topics)+'_filter_direction='+str(filter_direction)+'_filter_control='+str(filter_control)+'_vectors.pickle'
        vectors = extract_slda_features(filtered_data, model, lda_vectors_filename, d_type='dev')
    if BOW:
        bow_vectors= extract_bow_features(filtered_data, word2index, pca_model)
    if empath:
        vectors = extract_empath_features(filtered_data, data_type=data_type, filter_direction=filter_direction, 
                            filter_control=filter_control, d_type='train')
    
    X_dev, y_dev, post_vectors = format_features(DEV_FOLDERPATH, model=model, vectors=vectors, 
                                                          pca_model=pca_model, bow_vectors=bow_vectors,
                                                         data_type=data_type, num_topics=num_topics, d_type='dev', 
                                                         BOW=BOW, sLDA=sLDA, LDA=LDA, empath=empath,
                                                        filter_direction=filter_direction, filter_control=filter_direction)
        
    
    y_pred = predict_posts(p_clf, X_dev, y_dev, print_metrics=True)
    
    user_to_post_label, user_to_y = format_post_labels(filtered_data, vectors, y_pred, DEV_FOLDERPATH)
    
    for user_clf_type in user_clf_types:
        print('START: User classification post:', user_clf_type)
        predict_users(user_to_post_label, user_to_y, user_clf_type=user_clf_type)
    return model, vectors, pca_model, bow_vectors, word2index, p_clf

# Expert Wrapped

In [ ]:
def expert_wrapped(test_data_processed=True,  num_topics=40, data_type='filter_normal', 
                   post_clf_type='RbfSVM', user_clf_types=['Max'], p_clf='',
                   model='', vectors='', pca_model='', bow_vectors='', word2index='', 
                   filter_direction=False, filter_control=False,
                   BOW=False, sLDA=False, LDA=False, empath=False):
    # Make sure you have created TRAIN_FOLDERPATH and DEV_FOLDERPATH directories on local
    TEST_POSTPATH = './Data/expert/expert_posts.csv'
    TEST_LABELPATH = './Data/expert/expert.csv'
    TEST_FOLDERPATH = './Processing/crowd_processed_expert/'
    
    if test_data_processed:
        user_to_post_expert, post_to_metadata_expert, filtered_data_expert, sw_posts_expert, sw_timestamps_expert = dataloader.load_from_folder(TEST_FOLDERPATH)
    else:
        user_to_post_expert, post_to_words_expert, post_to_metadata_expert = dataloader.load_posts(TEST_POSTPATH, append_title = False)
        post_to_label_expert = dataloader.load_classification(TEST_LABELPATH, user_to_post_expert, post_to_words_expert, post_to_metadata_expert)
        filtered_data_expert, sw_posts_expert, sw_timestamps_expert = dataloader.filter_posts(post_to_label_expert, post_to_metadata_expert, filter_images = True)
        print(len(filtered_data_expert))
        filtered_data_expert = dataloader.filter_near_SW(filtered_data_expert, post_to_metadata_expert, sw_timestamps_expert)
        print(len(filtered_data_expert))

        filtered_data_expert = dataloader.filter_stopwords(filtered_data_expert)
        sw_posts_expert = dataloader.filter_stopwords(sw_posts_expert)


        dataloader.save_to_folder(TEST_FOLDERPATH, user_to_post_expert, post_to_metadata_expert, filtered_data_expert, sw_posts_expert, sw_timestamps_expert)
        
    print_label_size(filtered_data_expert)
    
    if sLDA:
        slda_vectors_filename = 'slda_'+data_type+'_'+str(num_topics)+'_filter_direction='+str(filter_direction)+'_filter_control='+str(filter_control)+'_vectors.pickle'
        vectors = extract_slda_features(filtered_data_expert, model, slda_vectors_filename, d_type='test')
    if LDA:
        lda_vectors_filename = 'lda_'+data_type+'_'+str(num_topics)+'_filter_direction='+str(filter_direction)+'_filter_control='+str(filter_control)+'_vectors.pickle'
        vectors = extract_slda_features(filtered_data_expert, model, lda_vectors_filename, d_type='test')
    if BOW:
        bow_vectors= extract_bow_features(filtered_data_expert, word2index, pca_model)
    if empath:
        vectors = extract_empath_features(filtered_data_expert, data_type=data_type, filter_direction=filter_direction, 
                            filter_control=filter_control, d_type='test')
        
    X_test, y_test, post_vectors = format_features(TEST_FOLDERPATH, model=model, vectors=vectors, 
                                                          pca_model=pca_model, bow_vectors=bow_vectors,
                                                         data_type=data_type, num_topics=num_topics, d_type='test', 
                                                         BOW=BOW, sLDA=sLDA, LDA=LDA, empath=empath,
                                                        filter_direction=filter_direction, filter_control=filter_direction)

    
    y_pred = predict_posts(p_clf, X_test, y_test, print_metrics=True)
    
    user_to_post_label, user_to_y = format_post_labels(filtered_data_expert, vectors, y_pred, TEST_FOLDERPATH)
    
    for user_clf_type in user_clf_types:
        print('START: User classification post:', user_clf_type)
        predict_users(user_to_post_label, user_to_y, user_clf_type=user_clf_type)
        

# sLDA + LogReg

In [ ]:
data_types = ['']
# feature_extraction =  [sLDA=False, LDA=False, BOW=False, empath=False]
post_clf_typs = ['LogReg', 'LinearSVM', 'RbfSVM', 'AdaBoost', 'RandomForest', 'MLP']
user_clf_types = ['Max', 'Threshold']
model, vectors, pca_model, bow_vectors, word2index, p_clf = wrapped(train_data_processed=True, sLDA=True, num_topics=40, post_clf_type=post_clf_types[0])

# sLDA + LinearSVM

In [ ]:
data_types = ['']
# feature_extraction =  [sLDA=False, LDA=False, BOW=False, empath=False]
post_clf_typs = ['LogReg', 'LinearSVM', 'RbfSVM', 'AdaBoost', 'RandomForest', 'MLP']
user_clf_types = ['Max', 'Threshold']
model, vectors, pca_model, bow_vectors, word2index, p_clf = wrapped(train_data_processed=True, sLDA=True, 
                                                                    num_topics=40, post_clf_type=post_clf_types[1])

# sLDA + RbfSVM

In [ ]:
data_types = ['']
# feature_extraction =  [sLDA=False, LDA=False, BOW=False, empath=False]
post_clf_typs = ['LogReg', 'LinearSVM', 'RbfSVM', 'AdaBoost', 'RandomForest', 'MLP']
user_clf_types = ['Max', 'Threshold']
model, vectors, pca_model, bow_vectors, word2index, p_clf = wrapped(train_data_processed=True, sLDA=True, 
                                                                    num_topics=40, post_clf_type=post_clf_types[2])

# sLDA + AdaBoost

In [ ]:
data_types = ['']
# feature_extraction =  [sLDA=False, LDA=False, BOW=False, empath=False]
post_clf_typs = ['LogReg', 'LinearSVM', 'RbfSVM', 'AdaBoost', 'RandomForest', 'MLP']
user_clf_types = ['Max', 'Threshold']
model, vectors, pca_model, bow_vectors, word2index, p_clf = wrapped(train_data_processed=True, sLDA=True, 
                                                                    num_topics=40, post_clf_type=post_clf_types[3])

# sLDA + Random Forest

In [ ]:
data_types = ['']
# feature_extraction =  [sLDA=False, LDA=False, BOW=False, empath=False]
post_clf_typs = ['LogReg', 'LinearSVM', 'RbfSVM', 'AdaBoost', 'RandomForest', 'MLP']
user_clf_types = ['Max', 'Threshold']
model, vectors, pca_model, bow_vectors, word2index, p_clf = wrapped(train_data_processed=True, sLDA=True, 
                                                                    num_topics=40, post_clf_type=post_clf_types[4])

# sLDA + Random MLP

In [ ]:
data_types = ['']
# feature_extraction =  [sLDA=False, LDA=False, BOW=False, empath=False]
post_clf_typs = ['LogReg', 'LinearSVM', 'RbfSVM', 'AdaBoost', 'RandomForest', 'MLP']
user_clf_types = ['Max', 'Threshold']
model, vectors, pca_model, bow_vectors, word2index, p_clf = wrapped(train_data_processed=True, sLDA=True, 
                                                                    num_topics=40, post_clf_type=post_clf_types[5])

# sLDA + BOW + Adaboost

In [ ]:
data_types = ['']
# feature_extraction =  [sLDA=False, LDA=False, BOW=False, empath=False]
post_clf_typs = ['LogReg', 'LinearSVM', 'RbfSVM', 'AdaBoost', 'RandomForest', 'MLP']
user_clf_types = ['Max', 'Threshold']
model, vectors, pca_model, bow_vectors, word2index, p_clf = wrapped(train_data_processed=True, sLDA=True, BOW=True 
                                                                    num_topics=40, post_clf_type=post_clf_types[5])

In [ ]:
expert_wrapped(test_data_processed=True, model=model, vectors=vectors, pca_model='', bow_vectors='',
               word2index='', p_clf=p_clf, BOW=False, sLDA=True, LDA=False, empath=False)

In [ ]:
data_types = ['']
# feature_extraction =  [sLDA=False, LDA=False, BOW=False, empath=False]
post_clf_typs = ['LogReg', 'LinearSVM', 'RbfSVM', 'AdaBoost', 'RandomForest', 'MLP']
user_clf_types = ['Max', 'Threshold']
model, vectors, pca_model, bow_vectors, word2index, p_clf = wrapped(train_data_processed=True, sLDA=True, BOW=True, num_topics=40, post_clf_type=post_clf_types[0])

expert_wrapped(test_data_processed=True, model=model, vectors=vectors, pca_model='', bow_vectors='',
               word2index='', p_clf=p_clf, BOW=True, sLDA=True, LDA=False, empath=False)

In [ ]:
data_types = ['']
# feature_extraction =  [sLDA=False, LDA=False, BOW=False, empath=False]
post_clf_typs = ['LogReg', 'LinearSVM', 'RbfSVM', 'AdaBoost', 'RandomForest', 'MLP']
user_clf_types = ['Max', 'Threshold']
model, vectors, pca_model, bow_vectors, word2index, p_clf = wrapped(train_data_processed=True, sLDA=False, LDA=False, BOW=True, empath=False, 
                                                                    num_topics=40, post_clf_type=post_clf_types[0])

expert_wrapped(test_data_processed=True, model=model, vectors=vectors, pca_model='', bow_vectors='',
               num_topics=40, post_clf_type=post_clf_types[0], user_clf_type=['Max'],
               word2index='', p_clf=p_clf, sLDA=False, LDA=False, BOW=True, empath=False)